In [1]:
pip install kafka-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 3.3 MB/s eta 0:00:00 MB/s eta 0:00:01:01
Note: you may need to restart the kernel to use updated packages.


In [39]:
import pandas as pd
from kafka import KafkaConsumer, KafkaProducer
from time import sleep
from functools import reduce
import json
import requests
import csv

In [6]:
producer = KafkaProducer(bootstrap_servers=["44.201.154.217:9092"],
                         value_serializer=lambda x: json.dumps(x).encode("utf-8"))

In [8]:
producer.send("mytopic", value="{'some_key': 1234}")

In [60]:
def save_data_to_csv(data):
    metadata_key, time_series_key = data.keys()
    csv_file_handler = open("./" + data[metadata_key]["2. Symbol"] + ".csv", "w")
    writer = csv.writer(csv_file_handler)
    time_series_data = data[time_series_key]
    
    # Write the CSV header
    writer.writerow(["Timestamp", "Open", "High", "Low", "Close", "Volume"])
    
    # Write values
    for key in time_series_data.keys():
        prices = time_series_data[key]
        price_attribute_keys = prices.keys()
        price_attributes_as_list = list(map(lambda key: prices[key], prices))
        writer.writerow([key] + price_attributes_as_list)

def extract_company_1_minute_intraday_stock_prices(company_tick):
    api_key = open("./api.key", "r").read()
    url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=' + company_tick + '&interval=1min&apikey=' + api_key
    data = requests.get(url).json()
    return data

In [61]:
data = extract_company_1_minute_intraday_stock_prices("GOOG")
save_data_to_csv(data)

In [62]:
df = pd.read_csv("GOOG.csv")
df.head()

,Timestamp,Open,High,Low,Close,Volume
0,2023-07-28 19:59:00,133.40,133.40,133.29,133.34,52
1,2023-07-28 19:58:00,133.30,133.34,133.26,133.34,664
2,2023-07-28 19:56:00,133.20,133.30,133.20,133.30,749
3,2023-07-28 19:55:00,133.21,133.22,133.21,133.22,4
4,2023-07-28 19:54:00,133.22,133.25,133.21,133.22,354


In [63]:
df.sample(1).to_dict(orient="records")

[{'Timestamp': '2023-07-28 19:50:00',
  'Open': 133.225,
  'High': 133.23,
  'Low': 133.225,
  'Close': 133.23,
  'Volume': 35}]